# Handling Bigger Incidents

## 1. Crashes in Complex Systems

Up to now we've talked about how to diagnose and fix errors that are confined to one computer. That's a common case for computers that are used by a single user. But once we start going into complex systems that involve many different services, we'll need to take a look at the bigger picture and have different computers interact with each other. 

Say you're in charge of the e-commerce site for your company. The page as seen by the users recently started responding with internal server error to about 20% of all requests. How do you figure out what's going on? You want to apply the same principles that we saw for troubleshooting a problem on one computer, but this time at a larger scale. So you'll want to check the log messages in the servers providing the service, and see if you find any additional information pointing to what's causing the issue. You'll want to find any log specific to the service that's failing, and also look at the general system logs to see if there's a problem affecting the server in general. 

For this example, let's say you find a bunch of entries in the logs that say, invalid response from server. That's not a great error message. You don't know what the request was or what the response was, but it's at least a clue that whatever's happening is related to some other service in the overall system. We said that this started failing recently, so it might make sense to figure out what changed between what it was working correctly and when it started to fail. Was there a new version of the system deployed? Were there any relevant changes regarding the requests? 

Let's say this is happening on a Tuesday morning, and the latest release of this service was the previous week. Things were working fine until today, and the requests seemed normal, nothing out of the ordinary. So the service itself is probably okay, but what about the other services involved in the system? Was there a new version of one of the underlying systems, like the database, the authentication service, or some other back-end server like the inventory, billing, or procurement systems?

Looking at recent changes, you see that there were a bunch of changes made earlier in the day to the load balancer used between the front-end and the back-end services. Since the only clue you have is that the response from the service was invalid, you're not sure that these changes are at fault, but they sure seem suspicious. Whenever possible, the best strategy is to roll back the changes that you suspect are causing the issue, even if you aren't 100% sure if this is the actual cause. 

If your infrastructure allows easy rollbacks, try that before doing any further investigation. Why? Because that way, you'll restore the service back to health if it was the cause, or you'll eliminate this change as a possible cause if doing the rollback doesn't help. Whether you do the rollback or not, when coming across unhelpful error messages, it's a good idea to improve them. Instead of the error just saying that the response is invalid, change it to include what the request and the response were, and why the response was invalid. That way, the next time you're trying to debug a similar issue you already have more information to work with. 

For this example, if the error had included this information you'd have seen that the invalid response was a 404 error. This was caused by having a server added to the pool as part of the inventory system, but the server actually belonged to the procurement system. Now, say a couple of weeks later you see that again, there are a bunch of internal server errors in the same service. It might be tempting to assume that it's the load balancer's fault once again, but by now you know that you should always look at the logs first and see what you find. There's no reason why the error should be the same this time. 

When looking at the logs you may notice, for example, that only one of the front-end servers is actually affected by the problem. All the other machines are serving their content successfully. In a case like this, you'd start by first removing the machine from the pool of servers that can provide this service. That way, you avoid users getting any more errors. Well, you can investigate what's going on with the broken machine. As you've probably realized by now, when dealing with complex systems like these having good logs is essential to understanding what's going on. On top of that, you'll want to have good monitoring of what the service is doing and use version control for all changes so that you can quickly check what's changed and roll back when needed. It's also important that you can very quickly deploy new machines when necessary. This could be achieved by either keeping standby servers, in case you need to use them, or by having a tested pipeline that allows you to deploy new servers on demand.